In [1]:
import numpy as np
from gym import Env
from gym.spaces import Box, Discrete
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

# Smart Charging with Reinforcement Learning

This notebook trains a deep Qnetwork agent to regulate the charging of an electric taxi in the time interval of two hours where the charging rate is adjusted every 15 minutes. The maximum charging rate per hour is set to 22kW. After the two hours the battery demand is calculated with a normal distribution with the mean 30kW and sigma 5kW. The charging cost formula is given by a time coefficient alpha, which we set to a constant value here, that is multiplied with e^power where power is the charging rate/4 because we calculate the costs for 15min intervals.

We used the following tutorial for our coding: https://www.section.io/engineering-education/building-a-reinforcement-learning-environment-using-openai-gym/

In [2]:
class ChargingEnv(Env):
    def __init__(self, max_charging_rate=22, mu=30, sigma=5, alpha_t=0.25):
        self.max_charging_rate = max_charging_rate
        self.battery_capacity = 100 # maximum battery capacity
        self.mu = mu
        self.sigma = sigma
        self.alpha_t = alpha_t
        self.action_space = Discrete(4) # 4 charging actions: zero, low, medium, high
        self.observation_space = Box(low=np.array([0]), high=np.array([self.battery_capacity]))
        self.reset_state() # reset battery
        self.charging_length = 120 # 120 minutes of charging
    
    def reset_state(self):
        self.state = 10 + random.randint(-5,5) # random initial battery between 5 and 15

    def step(self, action): 
        charging_rate = action * (self.max_charging_rate/3) # assuming highest rate is 22 per hour
        power = charging_rate * 0.25 # assuming 15min interval
        self.state += power  
        self.state = min(self.state, self.battery_capacity)
        self.charging_length -= 15 # subtract 15min from charging length

        # Checking if charging is done
        if self.charging_length <= 0: 
            done = True
        else:
            done = False

        # Calculate charging cost
        charging_cost = self.alpha_t * np.exp(power)  # as per the given formula
        
        # Calculating the reward
        if done:
            energy_demand = np.random.normal(self.mu, self.sigma)
            if self.state < energy_demand:
                reward = -10000  # Penalty for running out of energy
            else:
                # Reward is proportional to the energy saved
                reward = -charging_cost
        else:
            reward = -charging_cost
        
        
        
        # Setting the placeholder for info
        info = {}
        
        # Returning the step information
        return self.state, reward, done, info
    
    def reset(self):
        self.reset_state()
        self.charging_length = 120 
        return self.state

In [3]:
env = ChargingEnv()

states = env.observation_space.shape
actions = env.action_space.n

/Users/julialauten/git/aaa/.venv/lib/python3.9/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
model = Sequential()    
model.add(Dense(24, activation='relu', input_shape=states))
model.add(Dense(24, activation='relu'))
model.add(Dense(actions, activation='linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 4)                 100       
                                                                 
Total params: 748 (2.92 KB)
Trainable params: 748 (2.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
def build_agent(model, actions, warmup_steps=1000):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=warmup_steps, target_model_update=1)
    return dqn

In [6]:
dqn = build_agent(model, actions, warmup_steps=10000)
dqn.compile(Adam(learning_rate=0.01), metrics=['mae'])
dqn.fit(env, nb_steps=80000, visualize=False, verbose=1)

2023-08-14 15:20:34.346308: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-08-14 15:20:34.369958: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_2_1/bias/Assign' id:140 op device:{requested: '', assigned: ''} def:{{{node dense_2_1/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_2_1/bias, dense_2_1/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 80000 steps ...
Interval 1 (0 steps performed)
  147/10000 [..............................] - ETA: 10s - reward: -485.5162

/Users/julialauten/git/aaa/.venv/lib/python3.9/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-08-14 15:20:34.580918: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_2/BiasAdd' id:73 op device:{requested: '', assigned: ''} def:{{{node dense_2/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_2/MatMul, dense_2/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-08-14 15:20:34.589058: W tensorflow/c/c_api.cc:304] Operation '{name:'count_1/Assign' id:242 op device:{requested: '', assigned: ''} def:{{{node count_1/Assign}} = AssignVariableOp[_has_manual

10000/10000 [==============================] - 12s 1ms/step - reward: -749.0919
1250 episodes - episode_reward: -5992.735 [-10197.990, -11.196]

Interval 2 (10000 steps performed)
    2/10000 [..............................] - ETA: 33:38 - reward: -31.3683

2023-08-14 15:20:46.211692: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_2_1/BiasAdd' id:145 op device:{requested: '', assigned: ''} def:{{{node dense_2_1/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_2_1/MatMul, dense_2_1/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-08-14 15:20:46.296893: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_3/AddN' id:358 op device:{requested: '', assigned: ''} def:{{{node loss_3/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss_3/mul, loss_3/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-08-14 15:20:4

10000/10000 [==============================] - 34s 3ms/step - reward: -546.2721
1250 episodes - episode_reward: -4370.177 [-10256.286, -9.882] - loss: 1868916.460 - mae: 2923.452 - mean_q: -3443.063

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 35s 3ms/step - reward: -517.3915
1250 episodes - episode_reward: -4139.132 [-10257.600, -18.099] - loss: 1743536.500 - mae: 2568.784 - mean_q: -2996.502

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 36s 4ms/step - reward: -535.4517
1250 episodes - episode_reward: -4283.613 [-10264.503, -14.158] - loss: 1764440.250 - mae: 2325.369 - mean_q: -2650.052

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 37s 4ms/step - reward: -529.6931
1250 episodes - episode_reward: -4237.545 [-10307.679, -22.374] - loss: 1887557.375 - mae: 1976.213 - mean_q: -2114.220

Interval 6 (50000 steps performed)
10000/10000 [==============================] - 37s 4ms/st

In [7]:
results = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(results.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: -378.382, steps: 8
Episode 2: reward: -489.384, steps: 8
Episode 3: reward: -489.384, steps: 8
Episode 4: reward: -386.599, steps: 8
Episode 5: reward: -489.384, steps: 8
Episode 6: reward: -489.384, steps: 8
Episode 7: reward: -437.991, steps: 8
Episode 8: reward: -489.384, steps: 8
Episode 9: reward: -489.384, steps: 8
Episode 10: reward: -437.991, steps: 8
-457.72660271237527


## Testing in new Environment

After training and testing the agent we now create a similar environment that prints out the charging costs and battery stat for each 15min interval to get a closer look into how the agent behaves. 

In [8]:
class PrintEnv(Env):
    def __init__(self, max_charging_rate=22, mu=30, sigma=5, alpha_t=0.25):
        self.max_charging_rate = max_charging_rate
        self.battery_capacity = 100 # maximum battery capacity
        self.mu = mu
        self.sigma = sigma
        self.alpha_t = alpha_t
        self.action_space = Discrete(4) # 4 charging actions: zero, low, medium, high
        self.observation_space = Box(low=np.array([0]), high=np.array([self.battery_capacity]))
        self.reset_state() # reset battery
        self.charging_length = 120 # 120 minutes of charging
    
    def reset_state(self):
        self.state = 10 + random.randint(-5,5) # random initial battery between 5 and 15
        print("Initial battery level: ", self.state)

    def step(self, action): 
        charging_rate = action * (self.max_charging_rate/3) # assuming highest rate is 22 per hour
        power = charging_rate * 0.25 # assuming 15min interval
        self.state += power  
        self.state = min(self.state, self.battery_capacity)
        self.charging_length -= 15 # subtract 15min from charging length

        # Checking if charging is done
        if self.charging_length <= 0: 
            done = True
        else:
            done = False

        # Calculate charging cost
        charging_cost = self.alpha_t * np.exp(power)  # as per the given formula

        print("Time left: ", self.charging_length, "min | Power:", power.__round__(2), "| Battery level:", self.state.__round__(2), "| Charging cost:", charging_cost.__round__(2))
        
        # Calculating the reward
        if done:
            energy_demand = np.random.normal(self.mu, self.sigma)
            if self.state < energy_demand:
                reward = -10000  # Penalty for running out of energy
                print("Running out of energy!")
            else:
                # Reward is proportional to the energy saved
                reward = -charging_cost
                print("Charging done!")
        else:
            reward = -charging_cost
        
        
        
        # Setting the placeholder for info
        info = {}
        
        # Returning the step information
        return self.state, reward, done, info
    
    def reset(self):
        self.reset_state()
        self.charging_length = 120 
        return self.state

In [9]:
printEnv = PrintEnv()

num_episodes = 2
for episode in range(num_episodes):
    state = printEnv.reset()
    done = False
    total_reward = 0

    while not done:
        action = dqn.forward(state)
        next_state, reward, done, _ = printEnv.step(action)
        total_reward += reward
        state = next_state

    print(f"Episode {episode+1}: Total Reward = {total_reward}")



Initial battery level:  7
Initial battery level:  5
Time left:  105 min | Power: 1.83 | Battery level: 6.83 | Charging cost: 1.56
Time left:  90 min | Power: 3.67 | Battery level: 10.5 | Charging cost: 9.78
Time left:  75 min | Power: 5.5 | Battery level: 16.0 | Charging cost: 61.17
Time left:  60 min | Power: 5.5 | Battery level: 21.5 | Charging cost: 61.17
Time left:  45 min | Power: 5.5 | Battery level: 27.0 | Charging cost: 61.17
Time left:  30 min | Power: 5.5 | Battery level: 32.5 | Charging cost: 61.17
Time left:  15 min | Power: 5.5 | Battery level: 38.0 | Charging cost: 61.17
Time left:  0 min | Power: 5.5 | Battery level: 43.5 | Charging cost: 61.17
Charging done!
Episode 1: Total Reward = -378.381894633853
Initial battery level:  14
Time left:  105 min | Power: 5.5 | Battery level: 19.5 | Charging cost: 61.17
Time left:  90 min | Power: 5.5 | Battery level: 25.0 | Charging cost: 61.17
Time left:  75 min | Power: 5.5 | Battery level: 30.5 | Charging cost: 61.17
Time left:  60

We can see that the agent loads mostly with the highest rate of 22kW per hour (5.5kW in 15min) but starts with lower charging rates in the first episode.